## Description
* The Hansen et al. (2013) Global Forest Change dataset represents forest change, at 30 meters resolution, globally, during the years. More information can be found here <a href="https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2015_v1_3">Hansen Global Forest Change</a>. In this session, you will learn how to work with this data to analyse and quantifying forest change and how to calculate total forest area lost.

## Aims of the practical session
* Create ROI and load it
* Load the data and select the bands of interest
* Calculate and show both forest loss and gain
* Quantifying forest change

## Getting started

### Load packages

Import GEE packages that are needed for the analysis.

In [1]:
import ee
import geemap
# ee.Authenticate()

### Connect to Google Earth Engine (GEE)

Connect to the GEE to have access computing tools and GEE datasets.
You may be required to input your Google account for authorization.

In [2]:
Map = geemap.Map()
# Map.add_basemap('HYBRID')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Adding Region of Interest (ROI)

Create ROI that we want to work on it and then add and display it on the GEE map.
Import the downloaded shapefile for NSW from your computer path as ROI.

In [3]:
# # # load NSW shapefile
shp_path = 'D:/Project files/NSW shp files/NSW.shp'
geometry = geemap.shp_to_ee(shp_path)
Map.addLayer(geometry, {}, 'NSW SHP')
Map.centerObject(geometry);

### Displaying forest, loss, and gain
Use select() to create images and to make a visualization. 

In [4]:
# Displaying forest, loss, gain
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')
lossImage = gfc2014.select(['loss']).clip(geometry)
gainImage = gfc2014.select(['gain']).clip(geometry)
treeCover = gfc2014.select(['treecover2000']).clip(geometry)

In [5]:
# Add the tree cover layer in green.
Map.addLayer(treeCover.updateMask(treeCover),
    {'palette': ['000000',  '00FF00'], 'max': 100}, 'Forest Cover')

In [6]:
# Add the loss layer in red.
Map.addLayer(lossImage.updateMask(lossImage),
    {'palette': ['FF0000']}, 'Loss')

In [7]:
# Add the gain layer in blue.
Map.addLayer(gainImage.updateMask(gainImage),
    {'palette': ['0000FF']}, 'Gain');
Map

Map(bottom=10076.0, center=[-32.842673631954305, 147.30468750000003], controls=(WidgetControl(options=['positi…

<IPython.core.display.Javascript object>

### <a name="ex3"></a> Challenge 1

<div class="alert alert-block alert-danger">

- Use the and() method to create the lossAndGain image and displaying pixels where both loss and gain occur. 

More information: The Hansen et al. dataset has a band whose pixels are 1 where loss occurred and 0 otherwise (loss) and a band that is 1 where gain has occurred and a 0 otherwise (gain). To create a band where pixels in both the loss and the gain bands have a 1, you can use the and() logical method on images. The and() method is called like image1.and(image2) and returns an image in which pixels are 1 where both image1 and image2 are 1, and 0 elsewhere.
    
</div>    

### Quantifying Forest Change in a Region of Interest
To help compute areas, Earth Engine has the ee.Image.pixelArea() method which generates an image in which the value of each pixel is the pixel's area in square meters. Multiplying the loss image with this area image and then summing over the result gives us a measure of area.

In [8]:
# Get the forest loss image.
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')
lossImage = gfc2014.select(['loss'])
areaImage = lossImage.multiply(ee.Image.pixelArea())

In [9]:
# Sum the values of forest loss pixels in the ROI.
stats = areaImage.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': geometry,
  'scale': 30,
  'maxPixels': 1e13
})

In [10]:
classAreas = ee.List(stats.get('loss'))
print('pixels representing loss: ', classAreas.getInfo(), 'square meters')

pixels representing loss:  5620170645.178694 square meters


## References
This is where the references go. For exmaple:

* Hansen, M. C., Potapov, P. V., Moore, R., Hancher, M., Turubanova, S. A., Tyukavina, A., ... & Townshend, J. (2013). High-resolution global maps of 21st-century forest cover change. science, 342(6160), 850-853.

* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305

## Additional information

**License:** The code in this notebook was initially created by the team at [Digital Earth Australia](https://github.com/GeoscienceAustralia/dea-notebooks), and has been modified by Abolfazl Abdollahi. The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle site.

**Last modified:** June 2022

### Challenge answers

<a name="ex1answer">Answer to challenge 1</a>

In [5]:
# # Use the and() method to create the lossAndGain image.
# gainAndLoss = gainImage.And(lossImage)

In [9]:
# # Show the loss and gain image.
# Map.addLayer(gainAndLoss.updateMask(gainAndLoss),
#     {'palette': 'FF00FF'}, 'Gain and Loss');